In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import firedrake
from firedrake import inner, div, dx, ds, sqrt
import icepack, icepack.models, icepack.plot

# Synthetic ice stream

In this demo, we'll simulate the evolution of a grounded ice stream.
Ice streams are substantially more complicated than a floating ice shelf.
First, there are more fields.
For floating ice shelves, we only had to consider the velocity, thickness, and fluidity, but for ice streams there's also the surface elevation and bed friction.
Second, the position of the grounded line -- the contour across which the glacier is thin enough to go afloat -- can migrate in time.

The diagnostic equation for an ice stream is similar to that of a floating ice shelf, but for the addition of a term for basal friction:

$$\nabla\cdot hM - C|u|^{1/m - 1}u - \rho gh\nabla s = 0,$$

where $m$ is the *sliding exponent* and $C$ is the *sliding coefficient*.

The glacier state we'll start out with is grounded throughout the entire domain, and during the simulation it will thin out and go afloat part way through.
Accurately predicting grounding line migration is a major problem in glacier flow modeling.

### Geometry and input data

The geometry for this problem is based on that of Helheim Glacier in southeast Greenland.
The terminus of Helheim is about 5.2km wide while the inflow region of the Helheim catchment is roughly a semicircle with a radius of 30km.
The glacier is about 730m thick at the terminus and 1km thick around the inflow boundary, although in the main trunks the thickness is more like 1200-1500m.
Around the inflow boundary, the bed sits roughly 500-800m above sea level, while the fjord bottom is around 670m below sea level at the terminus.
Finally, much of the flow of Helheim is dictated by drag at the side walls and around a pair of rocky outcroppings or [nunataks](https://en.wikipedia.org/wiki/Nunatak).

In [ ]:
import numpy as np
from numpy import pi as π
import pygmsh
R = 30e3
δx = 2.5e3
θ = 15 * π / 48
ϕ = 14 * π / 48
t = .15

w = np.array([+1, 0, 0])
x = np.array([np.cos(θ), np.sin(θ), 0])
c = 2 * x
y = c + np.array([np.cos(π + θ - ϕ), np.sin(π + θ - ϕ), 0])
d = np.array([np.sin(π + θ - ϕ), -np.cos(π + θ - ϕ), 0])
z = y + t * d

xs = np.stack([w, x, y, z])

In [ ]:
geometry = pygmsh.built_in.Geometry()

center = geometry.add_point([0, 0, 0], lcar=δx)
center_east = geometry.add_point(R * c, lcar=δx)
points_east = [geometry.add_point(R * x, lcar=δx) for x in xs]
arc_east = [geometry.add_circle_arc(points_east[0], center, points_east[1]),
            geometry.add_circle_arc(points_east[1], center_east, points_east[2]),
            geometry.add_line(points_east[2], points_east[3])]

Q = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, 1]])
center_west = geometry.add_point(R * Q.dot(c), lcar=δx)
points_west = [geometry.add_point(R * Q.dot(x), lcar=δx) for x in xs]
arc_west = [geometry.add_circle_arc(points_west[1], center, points_west[0]),
            geometry.add_circle_arc(points_west[2], center_west, points_west[1]),
            geometry.add_line(points_west[3], points_west[2])]

inflow_center = geometry.add_point([0, 4 * R, 0], lcar=δx)
inflow_arc = geometry.add_circle_arc(points_west[0], inflow_center, points_east[0])
outflow_arc = geometry.add_line(points_east[-1], points_west[-1])

line_loop = geometry.add_line_loop(arc_east + [outflow_arc] + arc_west + [inflow_arc])
plane_surface = geometry.add_plane_surface(line_loop)
physical_lines = [geometry.add_physical(arc_east),
                  geometry.add_physical(arc_west),
                  geometry.add_physical(inflow_arc),
                  geometry.add_physical(outflow_arc)]
physical_surface = geometry.add_physical(plane_surface)

with open('ice-stream.geo', 'w') as geo_file:
    geo_file.write(geometry.get_code())
    
!gmsh -2 -format msh2 -o ice-stream.msh ice-stream.geo

In [ ]:
mesh = firedrake.Mesh('ice-stream.msh')

In [ ]:
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh)

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=2)

In [ ]:
x, y = firedrake.SpatialCoordinate(mesh)

b_in, b_out = 200, -400

As a sanity check, we'll evaluate the driving stress

$$\tau_D = -\rho_Igh\nabla s$$

at the inflow boundary of the ice stream to make sure the value isn't absurd.